# Testing of features in polymerist

In [ ]:
# Supressing annoying warnings (!must be done first!)
import warnings
warnings.catch_warnings(record=True)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# Logging
from polymerist.genutils.logutils.IOHandlers import LOG_FORMATTER

import logging
logging.basicConfig(
    level=logging.INFO,
    format =LOG_FORMATTER._fmt,
    datefmt=LOG_FORMATTER.datefmt,
    force=True
)
LOGGER = logging.getLogger(__name__)

# General
import re, json
from pathlib import Path
from shutil import copyfile

import numpy as np
import matplotlib.pyplot as plt

# Logging
from rich.progress import Progress, track
import logging

# Chemistry
from openmm.unit import nanometer, angstrom
from openff.toolkit import Topology, Molecule, ForceField
from openff.units import unit as offunit

from openff.interchange import Interchange
from openff.interchange.components import _packmol as packmol

from rdkit import Chem
import openeye

# Custom
import polymerist as ps2
from polymerist.genutils.decorators.functional import allow_string_paths, allow_pathlib_paths, optional_in_place

## Testing combinatorics utils

In [ ]:
from polymerist.genutils import iteration
from polymerist.maths.combinatorics import core, graphenum, sequences, partitions

In [ ]:
for n in range(20):
    print(*(core.binomial_coeff(n, k) for k in range(n + 1)), sep=' ')

In [ ]:
for n in range(20):
    print(*(round(core.stirling_first(n, k)) for k in range(n + 1)), sep=' ')

In [ ]:
for n in range(20):
    print(round(core.bell_number(n)))

In [ ]:
for n in range(10):
    print(n, iteration.iter_len(partitions.int_partitions(n)))

In [ ]:
for n in range(20):
    print(round(core.catalan_number(n)))

## Testing that reactant pathfinding for AnnotatedReaction works as intended

In [ ]:
from polymerist.smileslib.substructures import matching_labels_from_substruct_dict, matching_dict_from_substruct_dict
from polymerist.rdutils.reactions.reactions import AnnotatedReaction
from polymerist.rdutils import rdkdraw

rdkdraw.disable_substruct_highlights()

from rdkit import Chem

In [ ]:
# rxn = AnnotatedReaction.from_smarts("[#8:1](-[#0:2])-[H].[#17]-[#6:3](=[#8:4])-[#0:5]>>[#8:1](-[#0:2])-[#6:3](=[#8:4])-[#0:5]")
rxn = AnnotatedReaction.from_rxnfile('test.rxn')
reactant_templates_by_index = {i : reac_templ for i, reac_templ in enumerate(rxn.GetReactants())}


display(rxn)
print(reactant_templates_by_index)

In [ ]:
from polymerist.polymers.monomers.specification import expanded_SMILES, SANITIZE_AS_KEKULE

reactant_smiles = [
    'CC(O)C(Cl)C(Cl)CCO',
    'C1CCC(Cl)CC1O',
    'C1CCC(C(=O)Cl)CC1',
    'FC(F)(F)F',
    'c1(O)c(O)c(C(=O)Cl)c(O)c(C(=O)Cl)c1',
]

reactants = []
for smiles in reactant_smiles:
    exp_smiles = expanded_SMILES(smiles, assign_map_nums=False)
    mol = Chem.MolFromSmiles(exp_smiles, sanitize=False)
    Chem.SanitizeMol(mol, sanitizeOps=SANITIZE_AS_KEKULE)
    display(mol)
    reactants.append(mol)

In [ ]:
from itertools import combinations

for i, rs in enumerate(combinations(reactants, 2)):
    print(f'PAIR {i+1}')
    (react1, react2) = rs
    display(react1)
    display(react2)

    reactant_order = rxn.valid_reactant_ordering(rs)
    if reactant_order is not None:
        print('ORDER FOUND:')
        print('+'*50)
        for m in reactant_order:
            display(m)

    print('='*50)